In [12]:
import os
import json
import requests
from typing import Dict, Optional, Tuple
import time
import re
import random
from collections import defaultdict
import threading

# Set your DeepSeek API key here
DEEPSEEK_API_KEY = "sk-07507def2d2748b894125e3bf2******"

# Supported programming languages list
SUPPORTED_LANGUAGES = {
    "cpp": "C++",
    "java": "Java",
    "python": "Python",
    "python3": "Python3",
    "c": "C",
    "csharp": "C#",
    "javascript": "JavaScript",
    "typescript": "TypeScript",
    "php": "PHP",
    "swift": "Swift",
    "kotlin": "Kotlin",
    "dart": "Dart",
    "go": "Go",
    "ruby": "Ruby",
    "scala": "Scala",
    "rust": "Rust",
    "racket": "Racket",
    "erlang": "Erlang",
    "elixir": "Elixir",
    "cangjie": "Cangjie"
}

class LeetCodeSolutionGenerator:
    def __init__(self, api_key: str, language: str = "c"):
        """
        Initialize solution generator
        
        Args:
            api_key (str): DeepSeek API key
            language (str): Target programming language (default: "c")
        """
        if language not in SUPPORTED_LANGUAGES:
            raise ValueError(f"Unsupported language: {language}. Supported languages: {', '.join(SUPPORTED_LANGUAGES.keys())}")
            
        self.api_key = api_key
        self.language = language.lower()
        self.language_name = SUPPORTED_LANGUAGES[language]
        self.api_url = "https://api.deepseek.com/v1/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        
        # Create shared Session object (for better performance)
        self.session = requests.Session()
        self.session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=10, pool_maxsize=10))
        
        # Initialize statistics
        self.total_problems = 0
        self.generated_problems = 0
        self.error_problems = 0
        self.skipped_problems = 0
        self.problem_details = []
        
        # Thread lock for safe statistics updates
        self.lock = threading.Lock()
    
    def _read_problem(self, file_path: str) -> Dict:
        """Read and parse LeetCode problem JSON file"""
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    
    def _generate_prompt(self, problem_data: Dict) -> str:
        """Generate prompt for DeepSeek API based on problem data (optimized simplified version)"""
        title = problem_data.get('title', '')
        description = problem_data.get('description', '')
        
        # Simplified prompt to reduce API processing burden
        prompt = f"Please provide a C solution code for LeetCode problem '{title}'. Only provide code that can run directly on the LeetCode platform, without any explanations or comments.\n\nProblem Description:\n{description}"
        
        return prompt
    
    def _extract_code(self, solution: str) -> str:
        """Extract pure code from generated answer (enhanced cleaning logic)"""
        # Try to find code block boundaries
        code_start = -1
        code_end = -1
        
        # Find possible code block start markers (support multiple variants)
        start_patterns = [
            r"```\w*\n",        # Standard code block marker (```c\n)
            r"```\n",            # Code block without language marker (```\n)
        ]
        
        for pattern in start_patterns:
            match = re.search(pattern, solution)
            if match:
                code_start = match.end()
                break
        
        # Find possible code block end markers
        end_patterns = [
            r"\n```\s*$",        # Standard end marker
            r"\n```\s*\n",        # Standard end marker followed by newline
        ]
        
        for pattern in end_patterns:
            match = re.search(pattern, solution)
            if match:
                code_end = match.start()
                break
        
        # Extract main code section
        if code_start >= 0 and code_end >= 0 and code_end > code_start:
            solution = solution[code_start:code_end].strip()
        elif code_start >= 0:
            solution = solution[code_start:].strip()
        elif code_end >= 0:
            solution = solution[:code_end].strip()
        
        # Remove possible language identifier lines (more comprehensive patterns)
        solution = re.sub(r'^(//|#|--|<!--|/\*|\*)\s*(language|code|implementation|solution)\s*:\s*\w*\s*$', 
                         '', solution, flags=re.MULTILINE | re.IGNORECASE)
        
        # Remove extra blank lines
        solution = re.sub(r'\n\s*\n', '\n\n', solution)
        
        # Remove trailing explanatory text (if exists)
        solution = re.split(r'\n(?:This code|Code explanation|Explanation|note:|Note:)', solution, flags=re.IGNORECASE)[0]
        
        return solution.strip()
    
    def _call_deepseek_api(self, prompt: str) -> Tuple[Optional[str], Optional[str]]:
        """Call DeepSeek API to generate solution - using faster V3 model (optimized version)"""
        max_retries = 5  # Reduce maximum retry count (15→5)
        retry_count = 0
        error_details = []
        
        # Exponential backoff parameters (reduce delay time)
        base_delay = 2    # Reduce base delay (5→2)
        max_delay = 30    # Reduce maximum delay (120→30)
        
        while retry_count < max_retries:
            try:
                payload = {
                    "model": "deepseek-chat",  # Use faster V3 model
                    "messages": [
                        {"role": "user", "content": prompt}
                    ],
                    "temperature": 0.7,
                    "max_tokens": 2000
                }
                
                response = self.session.post(
                    self.api_url,
                    headers=self.headers,
                    json=payload,
                    timeout=(10, 80)  # Reduce timeout (20→10, 60→30)
                )
                
                if response.status_code == 200:
                    return response.json()['choices'][0]['message']['content'], None
                elif response.status_code == 429:  # Rate limit
                    # Increase wait time for rate limit errors
                    delay = 10 + random.uniform(0, 5)  # 10-15 seconds
                    print(f"Rate limit reached, waiting {delay:.1f} seconds before retry...")
                    time.sleep(delay)
                    continue
                else:
                    error_msg = f"HTTP error: {response.status_code} - {response.text[:100] if response.text else 'No response content'}"
                    print(f"Error calling DeepSeek API (attempt {retry_count+1}/{max_retries}): {error_msg}")
                    error_details.append(error_msg)
            
            except requests.exceptions.Timeout as e:
                error_msg = f"Request timeout: {str(e)}"
                print(f"DeepSeek API request timeout (attempt {retry_count+1}/{max_retries}): {error_msg}")
                error_details.append(error_msg)
            except requests.exceptions.ConnectionError as e:
                error_msg = f"Connection error: {str(e)}"
                print(f"DeepSeek API connection error (attempt {retry_count+1}/{max_retries}): {error_msg}")
                error_details.append(error_msg)
            except Exception as e:
                error_msg = f"Exception: {str(e)}"
                print(f"Exception occurred while calling DeepSeek API (attempt {retry_count+1}/{max_retries}): {error_msg}")
                error_details.append(error_msg)
            
            # Exponential backoff + random jitter
            delay = min(base_delay * (2 ** retry_count), max_delay)
            jitter = random.uniform(0, 1)  # Add random jitter to avoid synchronized retries
            sleep_time = delay + jitter
            
            print(f"Waiting {sleep_time:.2f} seconds before retry...")
            time.sleep(sleep_time)
            retry_count += 1
        
        final_error = f"DeepSeek API call failed, reached maximum retry count ({max_retries})\nError details:\n" + "\n".join(error_details)
        print(final_error)
        return None, final_error
    
    def _get_problem_slug(self, problem_file: str) -> str:
        """Extract problem slug from filename"""
        # Example: "1-two-sum.json" -> "two-sum"
        filename = os.path.basename(problem_file)
        # Remove extension
        filename = os.path.splitext(filename)[0]
        # Remove leading numbers and hyphen
        if '-' in filename:
            return filename.split('-', 1)[1]
        return filename
    
    def _solution_exists(self, problem_name: str, output_dir: str) -> bool:
        """Check if solution already exists"""
        # C uses .c extension
        solution_file = os.path.join(output_dir, f"{problem_name}.c")
        return os.path.exists(solution_file)
    
    def _process_single_problem(self, problem_file: str, output_dir: str):
        """Process single problem internally (supports concurrency)"""
        problem_name = os.path.splitext(os.path.basename(problem_file))[0]
        problem_details = {
            "problem": problem_name,
            "status": "Not processed",
            "details": "N/A"
        }
        
        try:
            # Check if solution already exists 
            if self._solution_exists(problem_name, output_dir):
                with self.lock:
                    self.skipped_problems += 1
                problem_details["status"] = "Already exists, skipped"
                print(f"Solution already exists, skipping problem: {problem_name}")
                return True
            
            # Read problem data
            problem_data = self._read_problem(problem_file)
            problem_title = problem_data.get('title', problem_name)
            
            # Generate prompt
            prompt = self._generate_prompt(problem_data)
            
            # Get solution from DeepSeek (with retry mechanism)
            solution, api_error = self._call_deepseek_api(prompt)
            
            if solution:
                # Extract pure code
                solution = self._extract_code(solution)
                
                # Create output directory if it doesn't exist
                os.makedirs(output_dir, exist_ok=True)
                
                # Generate output filename (C uses .c extension)
                output_file = os.path.join(output_dir, f"{problem_name}.c")
                
                # Save solution
                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write(solution)
                
                print(f"Successfully generated solution: {output_file}")
                with self.lock:
                    self.generated_problems += 1
                problem_details["status"] = "Generated successfully"
                
                return True
            else:
                error_msg = f"Failed to generate solution: {api_error if api_error else 'Unknown error'}"
                print(error_msg)
                with self.lock:
                    self.error_problems += 1
                problem_details["status"] = "Generation failed"
                problem_details["details"] = api_error if api_error else "DeepSeek API call failed"
                return False
                
        except Exception as e:
            import traceback
            error_msg = f"Error processing {problem_file}: {str(e)}\nException stack: {traceback.format_exc()}"
            print(error_msg)
            with self.lock:
                self.error_problems += 1
            problem_details["status"] = f"Processing exception: {str(e)}"
            problem_details["details"] = error_msg
            return False
        finally:
            with self.lock:
                self.problem_details.append(problem_details)
                self.total_problems += 1
                
                # Print current progress
                print(f"\nCurrent progress: {self.total_problems} problems")
                print(f"Generated: {self.generated_problems} problems, Errors: {self.error_problems} problems, Skipped: {self.skipped_problems} problems")
    
    def process_problem(self, problem_file: str, output_dir: str) -> bool:
        """Process single problem (wrapper method)"""
        return self._process_single_problem(problem_file, output_dir)
    
    def process_all_problems(self, problems_dir: str, output_dir: str, max_threads: int = 3) -> None:
        """
        Process all LeetCode problems in directory (generate solutions only)
        
        Args:
            problems_dir (str): Directory containing problem JSON files
            output_dir (str): Directory to save solutions
            max_threads (int): Maximum concurrent threads (default 3)
        """
        start_time = time.time()
        
        # Get problem file list
        problem_files = [f for f in os.listdir(problems_dir) if f.endswith('.json')]
        random.shuffle(problem_files)  # Randomize problem order
        total_files = len(problem_files)
        print(f"Found {total_files} problem files, will process in random order")
        
        # Create output directory (if it doesn't exist)
        os.makedirs(output_dir, exist_ok=True)
        
        # Use thread pool to process problems
        threads = []
        for i, filename in enumerate(problem_files):
            problem_file = os.path.join(problems_dir, filename)
            print(f"\n{'='*50}")
            print(f"Processing: {filename} ({i+1}/{total_files})")
            
            # Wait for available threads
            while len(threads) >= max_threads:
                # Clean up completed threads
                threads = [t for t in threads if t.is_alive()]
                if len(threads) >= max_threads:
                    time.sleep(0.1) 
                    continue
            
            # Create new thread to process problem
            t = threading.Thread(target=self._process_single_problem, args=(problem_file, output_dir))
            t.start()
            threads.append(t)
            time.sleep(0.1)
        
        # Wait for all threads to complete
        for t in threads:
            t.join()
        
        # Print statistics after processing completes
        self.print_statistics(start_time)
    
    def process_all_problems_sequential(self, problems_dir: str, output_dir: str) -> None:
        """
        Process all LeetCode problems in directory sequentially (no concurrency)
        
        Args:
            problems_dir (str): Directory containing problem JSON files
            output_dir (str): Directory to save solutions
        """
        start_time = time.time()
        
        # Get problem file list
        problem_files = [f for f in os.listdir(problems_dir) if f.endswith('.json')]
        random.shuffle(problem_files)  # Randomize problem order
        total_files = len(problem_files)
        print(f"Found {total_files} problem files, will process in random order")
        
        # Create output directory (if it doesn't exist)
        os.makedirs(output_dir, exist_ok=True)
        
        for i, filename in enumerate(problem_files):
            problem_file = os.path.join(problems_dir, filename)
            print(f"\n{'='*50}")
            print(f"Processing: {filename} ({i+1}/{total_files})")
            
            
            problem_name = os.path.splitext(filename)[0]
            if self._solution_exists(problem_name, output_dir):
                print(f"Solution already exists, skipping problem: {problem_name}")
                with self.lock:
                    self.skipped_problems += 1
                    self.total_problems += 1
                    self.problem_details.append({
                        "problem": problem_name,
                        "status": "Already exists, skipped",
                        "details": "N/A"
                    })
               
                continue
            
           
            self.process_problem(problem_file, output_dir)
            
           
            time.sleep(2)
        
        # Print statistics after processing completes
        self.print_statistics(start_time)
    
    def print_statistics(self, start_time: float) -> None:
        """Print statistics of processing results"""
        elapsed_time = time.time() - start_time
        minutes, seconds = divmod(elapsed_time, 60)
        
        print("\n" + "="*50)
        print("LeetCode C Solutions Generation Statistics Report")
        print("="*50)
        print(f"Total problems: {self.total_problems}")
        print(f"Generated problems: {self.generated_problems}")
        print(f"Skipped problems: {self.skipped_problems}")
        print(f"Error problems: {self.error_problems}")
        print(f"Total time: {int(minutes)} minutes {int(seconds)} seconds")
        print(f"Average speed: {self.total_problems / elapsed_time if elapsed_time > 0 else 'N/A':.2f} problems/minute")
        
        # Print error details
        error_problems = [p for p in self.problem_details if p['status'] not in ["Generated successfully", "Already exists, skipped"]]
        if error_problems:
            print("\nError problem details:")
            for problem in error_problems:
                print(f"\nProblem: {problem['problem']}")
                print(f"Status: {problem['status']}")
                if problem['details'] != "N/A":
                    print(f"Details: {problem['details']}")
        
        print("="*50)

def main():
    # Check if API key is set
    if DEEPSEEK_API_KEY == "Paste your API key here":
        print("Please set your API key in the DEEPSEEK_API_KEY variable at the beginning of the file")
        return
    
    # Initialize generator (using C language)
    generator = LeetCodeSolutionGenerator(
        api_key=DEEPSEEK_API_KEY,
        language="c"
    )
    
    # Process all problems
    problems_dir = "leetcode_data"
    output_dir = "c_solutions"  # Separate folder for C solutions
    
    try:
        # Use concurrent processing (use cautiously to avoid triggering API rate limits)
        # generator.process_all_problems(problems_dir, output_dir, max_threads=3)
        
        # Or use sequential processing (safer)
        generator.process_all_problems_sequential(problems_dir, output_dir)
    except KeyboardInterrupt:
        print("\nProgram interrupted, printing current statistics...")
        generator.print_statistics(time.time())
    except Exception as e:
        print(f"Program error: {str(e)}")
        generator.print_statistics(time.time())

if __name__ == "__main__":
    main()

Found 3011 problem files, will process in random order

Processing: 3402-minimum-operations-to-make-columns-strictly-increasing.json (1/3011)
Solution already exists, skipping problem: 3402-minimum-operations-to-make-columns-strictly-increasing

Processing: 3259-maximum-energy-boost-from-two-drinks.json (2/3011)
Solution already exists, skipping problem: 3259-maximum-energy-boost-from-two-drinks

Processing: 1383-maximum-performance-of-a-team.json (3/3011)
Solution already exists, skipping problem: 1383-maximum-performance-of-a-team

Processing: 2200-find-all-k-distant-indices-in-an-array.json (4/3011)
Solution already exists, skipping problem: 2200-find-all-k-distant-indices-in-an-array

Processing: 1486-xor-operation-in-an-array.json (5/3011)
Solution already exists, skipping problem: 1486-xor-operation-in-an-array

Processing: LCR 017-M1oyTv.json (6/3011)
Solution already exists, skipping problem: LCR 017-M1oyTv

Processing: 3485-longest-common-prefix-of-k-strings-after-removal.json